In [1]:
from dotenv import load_dotenv
import os
from pathlib import Path
from pyspark.sql import SparkSession

CURRENT_DIRECTORY_NOTEBOOK = None


def intitate_notebook():
    load_dotenv()
    global CURRENT_DIRECTORY_NOTEBOOK
    if CURRENT_DIRECTORY_NOTEBOOK is None:
        os.chdir(os.getenv("PROJECT_BASE_PATH"))
        CURRENT_DIRECTORY_NOTEBOOK = Path(os.getcwd())
        print("Current directory for notebook: ", CURRENT_DIRECTORY_NOTEBOOK)
    else:
        print(
            "Current directory for notebook is already set: ",
            CURRENT_DIRECTORY_NOTEBOOK,
        )


intitate_notebook()

Current directory for notebook:  /Users/shirshmall/Personal_Drive/Credit_Risk_Model_Project


In [2]:
from notebooks.utils import column_stats, get_shape_df, get_columns_high_missing_values
from tqdm import tqdm

In [3]:
parquet_file_path = CURRENT_DIRECTORY_NOTEBOOK / Path(
    "raw_main_dataset/home-credit-credit-risk-model-stability/parquet_files/train/train_static_0_0.parquet"
)

spark = SparkSession.builder.appName("Parquet_File_Exploration_Static_0").getOrCreate()

df_static_0 = spark.read.parquet(str(parquet_file_path))
df_static_0.show()

24/07/17 10:48:08 WARN Utils: Your hostname, Shirshs-MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 192.168.1.109 instead (on interface en0)
24/07/17 10:48:08 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/07/17 10:48:09 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/07/17 10:48:11 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+-------+-----------------------+------------------------------+------------+--------------------+-------------------+--------------------+---------------------+--------------------+--------------------+--------------------+-------------------------+------------------------+-------------------------+---------------------------+-------------------------+-------------------------+------------------------+------------------------------+----------------------+----------------+------------+----------------------+---------------------+---------------------+---------------+----------------+----------------+----------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+------------------------+------------------+------------------------+---------------+-------------+------------+--------------------------+--------------------+------------------------+-----------------------+-----------------------

In [4]:
parquet_file_path = CURRENT_DIRECTORY_NOTEBOOK / Path(
    "raw_main_dataset/home-credit-credit-risk-model-stability/parquet_files/train/train_static_0_1.parquet"
)

spark = SparkSession.builder.appName("Parquet_File_Exploration_Static_1").getOrCreate()

df_static_1 = spark.read.parquet(str(parquet_file_path))
df_static_1.show()

24/07/17 10:48:13 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


+-------+-----------------------+------------------------------+------------+--------------------+-------------------+--------------------+---------------------+--------------------+--------------------+--------------------+-------------------------+------------------------+-------------------------+---------------------------+-------------------------+-------------------------+------------------------+------------------------------+----------------------+----------------+------------+----------------------+---------------------+---------------------+---------------+----------------+----------------+----------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+------------------------+------------------+------------------------+---------------+-------------+------------+--------------------------+--------------------+------------------------+-----------------------+-----------------------

In [5]:
get_shape_df(df_static_0)
print(" - " * 20)
get_shape_df(df_static_1)
print(" - " * 20)
df_static_0.columns == df_static_1.columns

Number of rows: 1003757
Number of columns: 168
Shape of DataFrame: (1003757, 168)
 -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  - 
Number of rows: 522902
Number of columns: 168
Shape of DataFrame: (522902, 168)
 -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  - 


True

In [6]:
df_static_depth0 = df_static_0.union(df_static_1)

In [7]:
get_shape_df(df_static_depth0)

Number of rows: 1526659
Number of columns: 168
Shape of DataFrame: (1526659, 168)


In [8]:
missing_values_dict = get_columns_high_missing_values(
    df_static_depth0, threshold=20, display=True
)

100%|██████████| 168/168 [00:23<00:00,  7.30it/s]


In [9]:
df_static_depth0 = df_static_depth0.drop(*list(missing_values_dict.keys()))

In [10]:
df_static_depth0.show()

+-------+------------+--------------------+-------------------+--------------------+---------------------+--------------------+--------------------+--------------------+----------------------+---------------------+---------------------+---------------+----------------+----------------+----------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+-------------+------------+--------------------------+---------------------+-------------------------+--------------------+------------+--------+---------------+------------------------+-----------------+------------------+------------------------+--------------------------+-------------------------------+---------------------+---------------------------+------------------------------+---------------------+-------------------------------+-----------+-----------------+-------------------+--------------------------+----------------------+-----------

In [11]:
for column_name in tqdm(df_static_depth0.columns):
    column_stats(df_static_depth0, column_name)

  1%|▏         | 1/69 [00:02<02:32,  2.24s/it]                                  

Column Name: case_id
Minimum Value: 0
Maximum Value: 2703454
Missing Values Count: 0 (0.00%)
Number of Unique Values: 1526659
Total Number of Rows: 1526659
Column Name:  case_id
Description - Not Found
---------------------------------------------------------------------------------------------------- 




  3%|▎         | 2/69 [00:03<01:51,  1.66s/it]                                  

Column Name: annuity_780A
Minimum Value: 80.8
Maximum Value: 106007.0
Missing Values Count: 0 (0.00%)
Number of Unique Values: 75325
Total Number of Rows: 1526659
Column Name:  annuity_780A
Column Description:  Monthly annuity amount.
---------------------------------------------------------------------------------------------------- 




  4%|▍         | 3/69 [00:04<01:33,  1.42s/it]

...

Column Name: totaldebt_9A
Minimum Value: 0.0
Maximum Value: 1210629.1
Missing Values Count: 3 (0.00%)
Number of Unique Values: 363903
Total Number of Rows: 1526659
Column Name:  totaldebt_9A
Column Description:  Total amount of debt.
---------------------------------------------------------------------------------------------------- 




 99%|█████████▊| 68/69 [00:55<00:00,  1.00it/s]

Column Name: totalsettled_863A
Minimum Value: 0.0
Maximum Value: 48035036.0
Missing Values Count: 4 (0.00%)
Number of Unique Values: 747166
Total Number of Rows: 1526659
Column Name:  totalsettled_863A
Column Description:  Sum of all payments made by the client.
---------------------------------------------------------------------------------------------------- 




100%|██████████| 69/69 [00:55<00:00,  1.24it/s]

Column Name: twobodfilling_608L
Minimum Value: BO
Maximum Value: FO
Missing Values Count: 91 (0.01%)
Number of Unique Values: 2
Total Number of Rows: 1526659
Column Name:  twobodfilling_608L
Column Description:  Type of application process.
---------------------------------------------------------------------------------------------------- 




In [12]:
# for column in df_static_0.columns:
#     print("Column Name: ", column)
#     row_df = features_definitions_df[features_definitions_df['Variable'].str.contains(column)]
#     if row_df.shape[0]==0:
#         print("Description - Not Found")
#     elif row_df.shape[0]==1:
#         print("Description: ", features_definitions_df.iloc[row_df.index[0]]['Description'])
#     else:
#         raise Exception("More than one rows for Column: " + column)
#     print(" - "*20, "\n")

In [13]:
# for column in df_static_0.columns:
#     column_stats(df_static_0, column_name=column)

In [14]:
parquet_file_path = CURRENT_DIRECTORY_NOTEBOOK / Path(
    "raw_main_dataset/home-credit-credit-risk-model-stability/parquet_files/train/train_static_cb_0.parquet"
)

spark = SparkSession.builder.appName("Parquet_File_Exploration_Static_1").getOrCreate()

df_static_cb_0 = spark.read.parquet(str(parquet_file_path))
df_static_cb_0.show()

+-------+-------------------+-----------------------+-----------------------+--------------+---------------------+----------------+----------------+------------+------------+-----------+------------+-----------+--------------------+---------------+-------------+-----------------+--------------+--------------+--------------+-------------+-------------+-------------+----------------+---------------+---------------+--------------+-------------+------------+------------+------------+------------+------------+------------------+--------------+--------------+--------------------+-------------+-------------------+-------------------+-----------------+-----------------+-------------+--------------+-----------+--------------------+------------------+---------------------+---------------------+------------------+------------------+------------------+------------------+
|case_id|assignmentdate_238D|assignmentdate_4527235D|assignmentdate_4955616D|birthdate_574D|contractssum_5085716L|dateofbirth_33

In [15]:
column_stats(df_static_0, column_name="case_id")
column_stats(df_static_1, column_name="case_id")
column_stats(df_static_cb_0, column_name="case_id")

Column Name: case_id
Minimum Value: 0
Maximum Value: 2651092
Missing Values Count: 0 (0.00%)
Number of Unique Values: 1003757
Total Number of Rows: 1003757
Column Name:  case_id
Description - Not Found
---------------------------------------------------------------------------------------------------- 




Column Name: case_id
Minimum Value: 40626
Maximum Value: 2703454
Missing Values Count: 0 (0.00%)
Number of Unique Values: 522902
Total Number of Rows: 522902
Column Name:  case_id
Description - Not Found
---------------------------------------------------------------------------------------------------- 




Column Name: case_id
Minimum Value: 357
Maximum Value: 2703454
Missing Values Count: 0 (0.00%)
Number of Unique Values: 1500476
Total Number of Rows: 1500476
Column Name:  case_id
Description - Not Found
---------------------------------------------------------------------------------------------------- 




'Function - column_stats - Done'

In [16]:
missing_values_dict = get_columns_high_missing_values(
    df_static_cb_0, threshold=20, display=True
)

100%|██████████| 53/53 [00:03<00:00, 14.10it/s]
